In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. Bring out the data
season_stats = pd.read_csv("players_stats_by_season_with_position.csv")

X = season_stats[['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK']]
y = season_stats['position_number'].astype(int) - 1 

# 2. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Split the data to training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4. Convert to torch 
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# 5. 모델 정의
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(6, 32)  # 6 inputs
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)  # 5 Classes

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = SimpleNN()

# 6. 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. 
epochs = 50
for epoch in range(epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 8. Accuracy
with torch.no_grad():
    y_pred = model(X_test)
    predicted = torch.argmax(y_pred, axis=1)
    acc = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy on test set: {acc:.4f}')


Epoch [10/50], Loss: 1.5856
Epoch [20/50], Loss: 1.5310
Epoch [30/50], Loss: 1.4855
Epoch [40/50], Loss: 1.4461
Epoch [50/50], Loss: 1.4114
Accuracy on test set: 0.4414
